In [1]:
def read_transcript_from_id(transcript_id):

    path_to_data_folder = '/archive/shared/sim_center/shared/ameer/'
    # lookinto this dictionary to find the path
    # can also manually create the path and it would be faster but not by much
    merged_lookup = pd.read_csv(path_to_data_folder + 'grade_lookupv5.csv')
    transcript = ''

    path = merged_lookup[merged_lookup.id == transcript_id].path.iloc[0]
    with open(path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line != '\n':
            tok_line = line.split(' ')
            for i in range(len(tok_line)):
                transcript += ' ' + tok_line[i]


    return transcript

In [2]:
def reconstruct_transcript(path, id): 
    path = path + id + '.txt'
    with open(path, 'r') as file:
        lines = file.readlines()
 
    out = []
    
    for line in lines:
        if line.find(':')==-1: continue
        temp = {}
        temp['speaker'] = line[:line.find(':')]
        temp['text'] = line[line.find(':') + 1: ]
        out.append(temp)

    return out

In [3]:
def consolidate_transcript(transcript):
    
    new = []
    new.append({'speaker': transcript[0]['speaker'], 'text': transcript[0]['text']})
    
    for i in range(1, len(transcript)):
        this = {}
        if transcript[i]['speaker'] == transcript[i-1]['speaker']:
            new[-1]['text'] += transcript[i]['text']
        else:
            this['speaker'] = transcript[i]['speaker']
            this['text'] = transcript[i]['text']
            new.append(this)
    
    return new

In [4]:
def extract_text(transcript):
    out = ''
    for line in transcript:
        out += line['speaker'] + line['text']
    return out

In [5]:
id_set1 = ['01_0542_298135',
'02_0036_174595',
'03_0028_174553',
'04_0043_174686',
'05_0033_174804',
'06_0079_175106',
'07_0068_174641',
'08_0029_174576',
'09_0029_174582',
'10_0991_331330']

In [7]:
gold_path = '/archive/shared/sim_center/shared/annie/gpt4-gold-standard-diarized/'
gold_transcript = extract_text(consolidate_transcript(reconstruct_transcript(gold_path, id_set1[0])))
diar_path = '/archive/shared/sim_center/shared/annie/gpt4_3chunk_pass2/'
diar_transcript = extract_text(consolidate_transcript(reconstruct_transcript(diar_path, id_set1[0])))

In [18]:
diar_transcript

"ID 01_0542_298135\nInstructions Learners, you may begin your patient station. Please remove your cover sheet, jot down any notes, knock, and then enter. Thank you.\nPatient so so I don't know. Hello?\nStudent Hi, this is Mrs. Miller.\nPatient Yes, yes it is.\nStudent Hi Mrs Miller, my name is Jackson Agroz. I'm the medical student working at the clinic today. I am calling to speak about your daughter. What was your name again?\nPatient Jackson Agrroz.\nStudent Okay. Your S-47024. That's your OSCE number?\nPatient 427024, yes.\nStudent Okay, well, OSCENumber21, sorry. There we go, thank you. Well, I am glad you called. I was very worried because my daughter has had a cough for a long time, and it just seems it won't go away.\nPatient Okay. How long has it been going on for?\nStudent Five weeks is when it started.\nPatient Okay, and can you describe the cough to me?\nStudent It's a dry cough that's almost nonstop.\nPatient Okay?\nStudent And you said it's been getting worse over these f

In [10]:
import difflib

cases=[(gold_transcript, diar_transcript)] 

y = cases[0][0]
x = cases[0][1]

diff = difflib.SequenceMatcher(None, x, y)

for tag, x_start, x_end, y_start, y_end in diff.get_opcodes():
    if tag == 'insert':
        print(y[y_start:y_end].casefold())


patient 

patient 

patient


student okay. how long has it been going on for?

five weeks is when it started.
student 
patient it's a dry cough that's almost nonstop.

 do you think she'll need like an antibiotic or something?

patient yes. 

sick?
patient no. not that i know of.

 okay.

student 
okay.
student so bailey, she's seven years old, previously healthy. about five weeks ago, she had this initial kind 

student

student 
 okay, any other questions you have?
 thank your call. 
l 

patient
student
patient
 thank you.




In [17]:
x = cases[0][0]
y = cases[0][1]

diff = difflib.SequenceMatcher(None, x, y)

for tag, x_start, x_end, y_start, y_end in diff.get_opcodes():
    if tag == 'insert':
        print(y[y_start:y_end].casefold())

what was your name again?
student
 there we go, thank you. well, i am glad you called. i was very worried because my daughter has had a cough for a long time, and it just seems it won't go away.
five weeks ago she had had a fever and some nasal congestion and the cough, and it lasted two days. but the coughing is the only thing that didn't go away.
student okay. so was she seen in clinic at that time, or did she seem like her symptoms got better except for the cough?
patient 
,


l 
okay. 


no?
 does that sound agreeable to you? is that something that you can do to bring her in sometime? 
 yeah. okay, perfect.
 

patient

student

instructions




In [26]:
import difflib

def diff_strings(a: str, b: str, *, use_loguru_colors: bool = False) -> str:
    output = []
    matcher = difflib.SequenceMatcher(None, a, b)
    if use_loguru_colors:
        green = '<GREEN><black>'
        red = '<RED><black>'
        endgreen = '</black></GREEN>'
        endred = '</black></RED>'
    else:
        green = '\x1b[38;5;16;48;5;2m'
        red = '\x1b[38;5;16;48;5;1m'
        endgreen = '\x1b[0m'
        endred = '\x1b[0m'

    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == 'equal':
            output.append(a[a0:a1])
        elif opcode == 'insert':
            output.append(f'{green}{b[b0:b1]}{endgreen}')
        elif opcode == 'delete':
            output.append(f'{red}{a[a0:a1]}{endred}')
        elif opcode == 'replace':
            output.append(f'{green}{b[b0:b1]}{endgreen}')
            output.append(f'{red}{a[a0:a1]}{endred}')
    return ''.join(output)

print(diff_strings(cases[0][0], cases[0][1]))

from loguru import logger
logger.opt(raw=True, colors=True).info(diff_strings(cases[0][0], cases[0][1], use_loguru_colors=True))

ID 01_0542_298135
Instructions Learners, you may begin your patient station. Please remove your cover sheet, jot down any notes, knock, and then enter. Thank you.
Patient so so I don't know. Hello?
Student Hi, this is Mrs. Miller.
Patient Yes, yes it is.
Student Hi Mrs Miller, my name is Jackson Agroz. I'm the medical student working at the clinic today. I am calling to speak about your daughter. What was your name again?
Patient Jackson Agrroz.What was your name again?
Student Jackson Agrroz.
 Okay. Your S-47024. 
Patient That's your OSCE number?
PatiStudent 427024, yes.
Student Okay, well, OSCENumber21, sorry. There we go, thank you. Well, I am glad you called. I was very worried because my daughter has had a cough for a long time, and it just seems it won't go away.
Patient Okay. HoThere w long has it been going on for?
Student Five weeks is when it started.
Patient Okay, and can you describe the cough to me?
Student It's a dry cough that's almost nonstop.
Patient Okay?
Student And 

ID 01_0542_298135
Instructions Learners, you may begin your patient station. Please remove your cover sheet, jot down any notes, knock, and then enter. Thank you.
Patient so so I don't know. Hello?
Student Hi, this is Mrs. Miller.
Patient Yes, yes it is.
Student Hi Mrs Miller, my name is Jackson Agroz. I'm the medical student working at the clinic today. I am calling to speak about your daughter. What was your name again?
Patient Jackson Agrroz.What was your name again?
Student Jackson Agrroz.
 Okay. Your S-47024. 
Patient That's your OSCE number?
PatiStudent 427024, yes.
Student Okay, well, OSCENumber21, sorry. There we go, thank you. Well, I am glad you called. I was very worried because my daughter has had a cough for a long time, and it just seems it won't go away.
Patient Okay. HoThere w long has it been going on for?
Student Five weeks is when it started.
Patient Okay, and can you describe the cough to me?
Student It's a dry cough that's almost nonstop.
Patient Okay?
Student And 

In [19]:
label_dif_num = 14

gold_path = '/archive/shared/sim_center/shared/annie/gpt4-gold-standard-diarized/'
gold_transcript_lines = consolidate_transcript(reconstruct_transcript(gold_path, id_set1[0]))
diar_path = '/archive/shared/sim_center/shared/annie/gpt4_3chunk_pass2/'
diar_transcript_lines = consolidate_transcript(reconstruct_transcript(diar_path, id_set1[0]))

line_nums = len(gold_transcript_lines)
line_dif = line_nums - len(diar_transcript_lines)

In [20]:
print(line_nums)

84


In [21]:
print(line_dif)

3


In [23]:
84*2

168

In [24]:
14/168

0.08333333333333333

In [13]:
import difflib
from wasabi import color

def diff_strings(a, b):
    output = []
    matcher = difflib.SequenceMatcher(None, a, b)
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == "equal":
            output.append(a[a0:a1])
        elif opcode == "insert":
            output.append(color(b[b0:b1], fg=16, bg="green"))
        elif opcode == "delete":
            output.append(color(a[a0:a1], fg=16, bg="red"))
        elif opcode == "replace":
            output.append(color(b[b0:b1], fg=16, bg="green"))
            output.append(color(a[a0:a1], fg=16, bg="red"))
    return "".join(output)
    
print(diff_strings("helloo world", "hello world!"))

helloo world!


In [14]:
print(diff_strings(cases[0][0], cases[0][1]))

ID 01_0542_298135
Instructions Learners, you may begin your patient station. Please remove your cover sheet, jot down any notes, knock, and then enter. Thank you.
Patient so so I don't know. Hello?
Student Hi, this is Mrs. Miller.
Patient Yes, yes it is.
Student Hi Mrs Miller, my name is Jackson Agroz. I'm the medical student working at the clinic today. I am calling to speak about your daughter. What was your name again?
Patient Jackson Agrroz.What was your name again?
Student Jackson Agrroz.
 Okay. Your S-47024. 
Patient That's your OSCE number?
PatiStudent 427024, yes.
Student Okay, well, OSCENumber21, sorry. There we go, thank you. Well, I am glad you called. I was very worried because my daughter has had a cough for a long time, and it just seems it won't go away.
Patient Okay. HoThere w long has it been going on for?
Student Five weeks is when it started.
Patient Okay, and can you describe the cough to me?
Student It's a dry cough that's almost nonstop.
Patient Okay?
Student And 

In [15]:
import difflib

def diff(string_list, index_a=0, index_b=None, print_only=True):
    """
    Print or return a colour-coded diff of two items in a list of strings.
    Default: Compare first and last strings; print the output; return None.
    """
    index_b = index_b or len(string_list) -1
    green = '\x1b[38;5;16;48;5;2m'
    red = '\x1b[38;5;16;48;5;1m'
    end = '\x1b[0m'
    output = []
    string_a = string_list[index_a]
    string_b = string_list[index_b]
    matcher = difflib.SequenceMatcher(None, string_a, string_b)
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == "equal":
            output += [string_a[a0:a1]]
        elif opcode == "insert":
            output += [green + string_b[b0:b1] + end]
        elif opcode == "delete":
            output += [red + string_a[a0:a1] + end]
        elif opcode == "replace":
            output += [green + string_b[b0:b1] + end]
            output += [red + string_a[a0:a1] + end]
    output = "".join(output)
    if not print_only:
        return output
    print(f"\n{output}\n")

In [16]:
diff(cases[0])


ID 01_0542_298135
Instructions Learners, you may begin your patient station. Please remove your cover sheet, jot down any notes, knock, and then enter. Thank you.
Patient so so I don't know. Hello?
Student Hi, this is Mrs. Miller.
Patient Yes, yes it is.
Student Hi Mrs Miller, my name is Jackson Agroz. I'm the medical student working at the clinic today. I am calling to speak about your daughter. What was your name again?
Patient Jackson Agrroz.What was your name again?
Student Jackson Agrroz.
 Okay. Your S-47024. 
Patient That's your OSCE number?
PatiStudent 427024, yes.
Student Okay, well, OSCENumber21, sorry. There we go, thank you. Well, I am glad you called. I was very worried because my daughter has had a cough for a long time, and it just seems it won't go away.
Patient Okay. HoThere w long has it been going on for?
Student Five weeks is when it started.
Patient Okay, and can you describe the cough to me?
Student It's a dry cough that's almost nonstop.
Patient Okay?
Student And

In [27]:
14/(84+81)

0.08484848484848485